#Загрузка данных

In [ ]:
import re
import nltk
import contractions
import numpy as np
import unicodedata as ucd
import spacy
from corus import load_wiki
from nltk.corpus import stopwords
from num2words import num2words
from razdel import tokenize, sentenize
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from spacy import displacy
from gensim.utils import tokenize
from nltk import word_tokenize
from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger

In [ ]:
!pip install corus
!pip install razdel
!pip install contractions
!pip install num2words
!python3 -m spacy download ru_core_news_sm
!pip install natasha
!pip install pymystem3
!wget https://dumps.wikimedia.org/ruwiki/latest/ruwiki-latest-pages-articles.xml.bz2

In [ ]:
np.random.seed(42)
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stop_words = set(stopwords.words("russian"))

In [ ]:
path = '/content/ruwiki-latest-pages-articles.xml.bz2'
records = load_wiki(path)
next(records)

In [ ]:
dataset = [next(records).text.lower() for _ in range(1000)]
print(dataset[0])

# Предложения
sentences = [_.text for _ in list(sentenize(dataset[0]))]
print(sentences[0])

#Удаление стоп-слов

In [ ]:
def remove_stopwords(words):
    """
    Удаление стоп-слов из списка токенизированных слов
    """
    new_words = []

    for word in words:
        if word not in stop_words:
            new_words.append(word)
    return new_words

#Нормализация

In [ ]:
def replace_contractions(text):
    """
    Заменить сокращения в тексте
    """
    return contractions.fix(text)


sample = replace_contractions(dataset[0])
print(sample)
words = nltk.word_tokenize(sample)
print(words)

In [ ]:
def remove_non_ASCII(words):
    """
    Удаление символов, не являющихся символами ASCII, из списка токенизированных слов
    """
    new_words = []

    for word in words:
      new_word = ucd.normalize('NFKD', word)
      new_words.append(new_word)
    return new_words


def to_lowercase(words):
    """
    Преобразование всех символов в строчные из списка токенизированных слов
    """
    new_words = []

    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """
    Удаление знаков препинания из списка токенизированных слов
    """
    new_words = []

    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """
    Заменить все числа в списке токенизированных слов текстовым представлением
    """
    new_words = []

    for word in words:
        if word.isdigit():
            new_word = num2words(word, lang='ru')
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_eng_words(words):
    """
    Удаление английских символов, слов и небольшая очистка
    """
    substitute = []
    new_words = []
    substitute_1 = re.sub(u"[a-zA-Z]", " ", ' '.join(words))
    substitute_2 = re.sub(r"(\d)+.", "\1", substitute_1)
    substitute_3 = re.sub(r"[\s]\s+", "", substitute_2)
    new_words = re.sub(r"", "", substitute_3)
    return new_words

def normalize(words):
    """
    Нормализация слов
    """
    words = remove_non_ASCII(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    words = remove_eng_words(words)
    return words

words = nltk.word_tokenize(sample)
words = normalize(words)
print(words)

#Лемматизация/Стемминг

In [ ]:
def stem_words(language, words):
    """
    Стемминг в списке токенизированных слов
    """
    stemmer = SnowballStemmer(language)
    stems = []

    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """
    Лемматизировать глаголы в списке токенизированных слов
    """
    lemmatizer = WordNetLemmatizer()
    lemmas = []

    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    stems = stem_words('russian', words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas

last_words = nltk.word_tokenize(words)
stems, lemmas = stem_and_lemmatize(last_words)
print('Исходные лексемы:\n', stems)
print('Результат лемматизации:\n', lemmas)

In [ ]:
sentences = sent_tokenize(words, language='russian')
nlp = spacy.load("ru_core_news_sm")
doc = nlp(words)
cnt = 0

for token in doc:
    if cnt == 10:
        break
    print(token.text, token.dep_, token.head)
    cnt += 1

In [ ]:
displacy.render(doc[2:13], style='dep', jupyter=True)

In [ ]:
# gensim токены
gen_tokens = [word for word in tokenize(words)]
print(f'gensim: {gen_tokens[25:35]}')

# Использование лемматизатора WordNet
wnl = WordNetLemmatizer()
word_net_lemmas = [(word, wnl.lemmatize(word)) for word in gen_tokens]
print(f'\nWordNet: {word_net_lemmas[5:24]}\n')

# Использование Razdel + Natasha
doc = Doc(words)
segmenter = Segmenter()
doc.segment(segmenter)
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
doc.tag_morph(morph_tagger)
print(f'Razdel + Natasha: {doc.tokens[:5]}')
print(doc.sents[:5])

# Использование библиотеки pymystem3
m = Mystem()
lemmas = m.lemmatize(words)
print(f'\npymystem3: {lemmas[5:15]}')
print(m.analyze(words)[5:10])